In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# -------------------------
# 1. Device configuration
# -------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# -------------------------
# 2. Dataset & Dataloader
# -------------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # mean & std of MNIST
])

train_dataset = datasets.MNIST(root="../data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="../data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

# -------------------------
# 3. Simple CNN Model
# -------------------------
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# -------------------------
# 4. Training Loop
# -------------------------
EPOCHS = 3

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# -------------------------
# 5. Evaluation
# -------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# -------------------------
# 6. Save the clean model
# -------------------------
torch.save(model.state_dict(), "../data/clean_cnn.pth")
print("Model saved as clean_cnn.pth")


Using device: cpu


Epoch 1/3: 100%|█████████████████████████████████████████████████████████████████████| 938/938 [00:13<00:00, 69.09it/s]


Epoch 1, Loss: 0.1361


Epoch 2/3: 100%|█████████████████████████████████████████████████████████████████████| 938/938 [00:14<00:00, 66.58it/s]


Epoch 2, Loss: 0.0411


Epoch 3/3: 100%|█████████████████████████████████████████████████████████████████████| 938/938 [00:14<00:00, 65.26it/s]


Epoch 3, Loss: 0.0267
Test Accuracy: 99.00%
Model saved as clean_cnn.pth
